In [1]:
import pipeline.pipeline as pipeline
import db.knowhere_db as kdb
import time

In [2]:
fName = r'C:/Users/William/Documents/DataScience/Projects/Knowhere/data/aggregated_emil_sensor2.csv'

In [7]:
import glob
from re import search

filelist = glob.glob('C:/Users/William/Documents/DataScience/Projects/Knowhere/data/iphone/Emil*')
with open(fName, 'w+') as af:
    af.write('timestamp, sensor, data_name, data_display, data_raw\n')
    for fn in filelist:
        with open(fn, 'r+') as f:
            next(f)
            for line in f:
                if not search('Screen', line):
                    af.write(line)


In [3]:
user_df = pipeline.read_csv(file_with_path=fName, user_id='58c5b91d71a9e82f33ad1668')

In [10]:
user_df

,sensor,data_name,data_display,data_raw,user_id
timestamp,,,,,
2017-03-08 10:38:48,GPS,Latitude,40.75303927551986,40.7530392755199,58c5b91d71a9e82f33ad1668
2017-03-08 10:38:48,GPS,Longitude,-73.99349961060088,-73.9934996106009,58c5b91d71a9e82f33ad1668
2017-03-08 10:38:48,GPS,Altitude,77.87607574462891 Meters,77.8760757446289,58c5b91d71a9e82f33ad1668
2017-03-08 10:38:48,GPS,Horizontal Accuracy,200 Meters,200.0,58c5b91d71a9e82f33ad1668
2017-03-08 10:38:48,GPS,Vertical Accuracy,17.09582975522192 Meters,17.0958297552219,58c5b91d71a9e82f33ad1668
2017-03-08 10:38:49,Cell Radio,Radio Access Technology,4G (LTE),4G (LTE),58c5b91d71a9e82f33ad1668
2017-03-08 10:38:49,Cell Radio,Carrier,Verizon,Verizon,58c5b91d71a9e82f33ad1668
2017-03-08 10:38:49,Proximity,Active,No,false,58c5b91d71a9e82f33ad1668
2017-03-08 10:38:49,Battery,Level,55%,0.55,58c5b91d71a9e82f33ad1668


In [4]:
user_df = pipeline.clean_iphone_data(user_df)

In [9]:
def agg_data(x):
    agg = {}
    for d,r in zip(x.data_name, x.data_raw):
        agg[d] = r
    return agg


import pandas as pd
from bson import ObjectId
df = user_df
df = df.filter(items=['user_id', 'sensor', 'data_name', 'data_raw'])
df = df.groupby([df.index, 'user_id','sensor']).agg(lambda x: tuple(x))
df = df.reset_index(level=['user_id', 'sensor'])
df['data'] =df.apply(lambda row: agg_data(row), axis=1)
df = df.reset_index()
df = df.filter(items=['timestamp', 'user_id', 'sensor', 'data'])
df.timestamp = pd.to_datetime(df.timestamp)
df.user_id = df.user_id.apply(ObjectId)
df

,timestamp,user_id,sensor,data
0,2017-03-08 10:38:48,58c5b91d71a9e82f33ad1668,GPS,"{u'Latitude': u'40.7530392755199', u'Vertical ..."
1,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668,Acceleration,"{u'y': u'0.0504199266433716', u'x': u'-0.00513..."
2,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668,Altimeter (Barometer),"{u'Pressure': u'101.2276611328125', u'Relative..."
3,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668,Attitude,"{u'Yaw': u'-0.422258717009882', u'Quarternion ..."
4,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668,Battery,"{u'State': u'Discharging', u'Level': u'0.55'}"
5,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668,Bluetooth,{u'Status': u'Off'}
6,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668,Cell Radio,"{u'Radio Access Technology': u'4G (LTE)', u'Ca..."
7,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668,GPS,"{u'Latitude': u'40.7530392777554', u'Vertical ..."
8,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668,Gravity,"{u'y': u'-0.226155400276184', u'x': u'0.081479..."
9,2017-03-08 10:38:49,58c5b91d71a9e82f33ad1668,Gyrometer,"{u'y': u'-1.38406085968018', u'x': u'-0.787979..."


In [5]:
user_df = pipeline.aggregate_iphone_data(user_df)

In [6]:
user_df.to_dict(orient='records')

[{'data': {'Altitude': '77.8760757446289',
   'Horizontal Accuracy': '200.0',
   'Latitude': '40.7530392755199',
   'Longitude': '-73.9934996106009',
   'Vertical Accuracy': '17.0958297552219'},
  'sensor': 'GPS',
  'user_id': ObjectId('58c5b91d71a9e82f33ad1668')},
 {'data': {'x': '-0.00513987150043249',
   'y': '0.0504199266433716',
   'z': '-0.0511120483279228'},
  'sensor': 'Acceleration',
  'user_id': ObjectId('58c5b91d71a9e82f33ad1668')},
 {'data': {'Pressure': '101.2276611328125', 'Relative Altitude': '0'},
  'sensor': 'Altimeter (Barometer)',
  'user_id': ObjectId('58c5b91d71a9e82f33ad1668')},
 {'data': {'Pitch': '0.228129006016354',
   'Quarternion w': '0.971587819598087',
   'Quarternion x': '0.119907817271495',
   'Quarternion y': '0.0168334498612349',
   'Quarternion z': '-0.203361400299109',
   'Roll': '0.0837446291126655',
   'Yaw': '-0.422258717009882'},
  'sensor': 'Attitude',
  'user_id': ObjectId('58c5b91d71a9e82f33ad1668')},
 {'data': {'Level': '0.55', 'State': 'Disch

In [8]:
writer = kdb.Writer('knowhere')
writer.write_dataframe_to_collection(df=user_df, collection='iphone_test')

In [7]:
reader = kdb.Reader('knowhere')
reader.get_dataframe(collection='iphone_test')

""
